## P0 Scanner
#### Original Author: Emil Sekerinski, revised February 2021

### Revised by group 14,  March 2021
##### Modifications:

* 1. DOUBLE and FLOAT are encoded by integer constants, add symbols `DOUBLE`, `FLOAT`, 
`CONVERT_TO_INT`, `CONVERT_TO_FLOAT`, `CONVERT_TO_DOUBLE`, `SUM`, `MAX`, `MIN`, `SQRT`, `RIGHTFLOOR`,`LEFTFLOOR`, `RIGHTCEIL`,
`LEFTCEIL`, `ABSVALUE`
* 2. Add KEYWORDS `sum`, `max`, `min`
* 3. Modify `getSym()`  to recogonize more unicode symbols `√`, `⎣`, `⎦`, `⎡`, `⎤`and `|`
* 4. Modify `number()` to recogonize a number and distinguish between integer, double, and float, and stores its value

In [1]:
TIMES = 1; DIV = 2; MOD = 3; AND = 4; PLUS = 5; MINUS = 6
OR = 7; EQ = 8; NE = 9; LT = 10; GT = 11; LE = 12; GE = 13
PERIOD = 14; COMMA = 15; COLON = 16; NOT = 17; LPAREN = 18
RPAREN = 19; LBRAK = 20; RBRAK = 21; LARROW = 22; RARROW = 23
LBRACE = 24; RBRACE = 25; CARD = 26; COMPLEMENT = 27; UNION = 28
INTERSECTION = 29; ELEMENT = 30; SUBSET = 31; SUPERSET = 32
DOTDOT = 33; THEN = 34; DO = 35; BECOMES = 36; INTEGER = 37
IDENT = 38; SEMICOLON = 39; ELSE = 40; IF = 41; WHILE = 42
CONST = 43; TYPE = 44; VAR = 45; SET = 46; PROCEDURE = 47
PROGRAM = 48; INDENT = 49; DEDENT = 50; EOF = 51; DOUBLE = 52; FLOAT = 53
CONVERT_TO_INT = 54; CONVERT_TO_FLOAT = 55; CONVERT_TO_DOUBLE = 56; SUM = 57; MAX = 58
SQRT = 59; RIGHTFLOOR = 60; LEFTFLOOR = 61; RIGHTCEIL = 62;
LEFTCEIL = 63; ABSVALUE = 64; MIN = 65

In [ ]:
def init(src):
    global line, lastline, pos, lastpos
    global ch, sym, val, source, index, indents
    line, lastline = 0, 1
    pos, lastpos = 1, 1
    ch, sym, val, source, index = '\n', None, None, src, 0
    indents = [1]; getChar(); getSym()

In [ ]:
def getChar():
    global line, lastline, pos, lastpos, ch, index
    if index == len(source): ch, index, pos = chr(0), index + 1, 1
    else:
        lastpos = pos
        if ch == '\n':
            pos, line = 1, line + 1
        else:
            lastline, pos = line, pos + 1
        ch, index = source[index], index + 1

In [ ]:
def mark(msg):
    raise Exception('line ' + str(lastline) + ' pos ' + str(lastpos) + ' ' + msg)

Procedure `number()` parses

    number ::= digit {digit} | digit {digit} '.' digit {digit} ['f'|'F']
    digit ::= '0' | ... | '9'

If the number fits in 32 bits, sets `sym` to `INTEGER` ; if the number is a decimal, end with `f` or `F` and fits in 32 bits floating-point number, set 'sym' to `FLOAT`; if the number is a decimal and fits in 64 bits floating-point number range, set `sym` to `DOUBLE`; set the value of the number to `val`

In [ ]:
def number():
    global sym, val, pos, index, lastpos, ch
    val = 0
    while '0' <= ch <= '9':
        val = 10 * val + int(ch)
        getChar()
    #  if the current character is ., check if next character is number, if not, just go back current position and recoginize the 
    # previous tokens as integer; if next character is a number, try to recogonize the tokens as float or double type number
    if ch == '.':
        getChar()
        if ch == '.':
            sym = INTEGER
            pos = pos - 2; index = index - 2; lastpos = pos; getChar() 
            if val >= 2**31:
                mark('integer too large'); val = 0 
        elif ch < '0' or ch > '9':
            mark('invalid floating-point number'); val = 0
        # regonize the dicimal places of the real number
        else:
            count = 0
            while '0' <= ch <= '9':
                count += 1
                val = val + int(ch)/10**count
                getChar()
            # if the number end with character f or F, it is a float number
            if ch == 'f' or ch == 'F':
                sym = FLOAT; getChar()
                if val >= 3.402824e+38:
                    mark('float too large'); val = 0
            else: 
                sym = DOUBLE; 
                if val >= 1.797694e+308:
                    mark('double too large'); value = 0
    else:
        sym = INTEGER
        if val >= 2**31:
            mark('integer too large'); val = 0

In [ ]:
KEYWORDS = \
    {'div': DIV, 'mod': MOD, 'and': AND, 'or': OR, 'then': THEN, 'do': DO,
     'else': ELSE, 'if': IF, 'while': WHILE, 'const': CONST, 'type': TYPE,
     'var': VAR, 'set': SET, 'procedure': PROCEDURE, 'program': PROGRAM,
     'sum': SUM, 'max': MAX, 'min': MIN}

def identKW():
    global sym, val
    start = index - 1
    while ('A' <= ch <= 'Z') or ('a' <= ch <= 'z') or \
          ('0' <= ch <= '9'): getChar()
    val = source[start:index-1]
    sym = KEYWORDS[val] if val in KEYWORDS else IDENT


In [ ]:
def comment():
    if ch == '/': getChar()
    else: mark('// expected')
    while chr(0) != ch != '\n': getChar()

In [ ]:
def getSym():
    global sym, indents, newline
    if pos < indents[0]:
        indents = indents[1:]; sym = DEDENT
    else:
        while ch in ' /':
            if ch == ' ': getChar() 
            else: comment()
        if ch == '\n': 
            while ch == '\n':
                getChar()
                while ch in ' /':
                    if ch == ' ': getChar() 
                    else: comment()
            if pos < indents[0]: sym, indents = DEDENT, indents[1:]; return
            elif pos > indents[0]: sym, indents = INDENT, [pos] + indents; return
        newline = pos == indents[0]
        if 'A' <= ch <= 'Z' or 'a' <= ch <= 'z': identKW()
        elif '0' <= ch <= '9': number()
        elif ch == '×': getChar(); sym = TIMES
        elif ch == '+': getChar(); sym = PLUS
        elif ch == '-': getChar(); sym = MINUS
        elif ch == '=': getChar(); sym = EQ
        elif ch == '≠': getChar(); sym = NE
        elif ch == '<': getChar(); sym = LT
        elif ch == '≤': getChar(); sym = LE
        elif ch == '>': getChar(); sym = GT
        elif ch == '≥': getChar(); sym = GE
        elif ch == ';': getChar(); sym = SEMICOLON
        elif ch == ',': getChar(); sym = COMMA
        elif ch == ':':
            getChar()
            if ch == '=': getChar(); sym = BECOMES
            else: sym = COLON
        elif ch == '.':
            getChar();
            if ch == '.': getChar(); sym = DOTDOT
            else: sym = PERIOD
        elif ch == '¬': getChar(); sym = NOT
        elif ch == '(': getChar(); sym = LPAREN
        elif ch == ')': getChar(); sym = RPAREN
        elif ch == '[': getChar(); sym = LBRAK
        elif ch == ']': getChar(); sym = RBRAK
        elif ch == '←': getChar(); sym = LARROW
        elif ch == '→': getChar(); sym = RARROW
        elif ch == '{': getChar(); sym = LBRACE
        elif ch == '}': getChar(); sym = RBRACE
        elif ch == '#': getChar(); sym = CARD
        elif ch == '∁': getChar(); sym = COMPLEMENT
        elif ch == '∪': getChar(); sym = UNION
        elif ch == '∩': getChar(); sym = INTERSECTION
        elif ch == '∈': getChar(); sym = ELEMENT
        elif ch == '⊆': getChar(); sym = SUBSET
        elif ch == '⊇': getChar(); sym = SUPERSET
        # recoginize new adding unicode symbols
        # Square root, Floor, Ceiling, Absolute value
        elif ch == '√': getChar(); sym = SQRT 
        elif ch == '⎣': getChar(); sym = LEFTFLOOR
        elif ch == '⎦': getChar(); sym = RIGHTFLOOR
        elif ch == '⎡': getChar(); sym = LEFTCEIL
        elif ch == '⎤': getChar(); sym = RIGHTCEIL
        elif ch == '|': getChar(); sym = ABSVALUE
        elif ch == chr(0): sym = EOF
        else: mark('illegal character')